In [1]:
!pip install pytorch-crf seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=9320db8a16d4d295874fa40042e8fb3faa8d5b8a96c040d882fda74ceffdbc3b
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [17]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torchcrf import CRF

In [5]:
# used the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ 使用設備: {device}")

✅ 使用設備: cuda


In [6]:
# read BIO tags
def read_bio_data(file_path):
    sentences, labels = [], []
    with open(file_path, 'r', encoding='utf-8') as f:
        sentence, label = [], []
        for line in f:
            line = line.strip()
            if line:
                parts = line.split("\t")
                if len(parts) == 2:
                    token, tag = parts
                    sentence.append(token)
                    label.append(tag)
            else:
                if sentence:
                    sentences.append(sentence)
                    labels.append(label)
                    sentence, label = [], []
    return sentences, labels

# read the test dataset
def read_test_data(file_path):
    sentences = []
    with open(file_path, 'r', encoding='utf-8') as f:
        sentence = []
        for line in f:
            line = line.strip()
            if line:
                sentence.append(line)
            else:
                if sentence:
                    sentences.append(sentence)
                    sentence = []
    return sentences

In [9]:
# data path
data_path = "/content/drive/MyDrive/A2-data/A2-data"

# read dataset
train_sentences, train_labels = read_bio_data(f"{data_path}/train")
dev_sentences, dev_labels = read_bio_data(f"{data_path}/dev.answers")
test_sentences = read_test_data(f"{data_path}/test")

print(f"✅ 訓練數據量: {len(train_sentences)}")
print(f"✅ 開發數據量: {len(dev_sentences)}")
print(f"✅ 測試數據量: {len(test_sentences)}")

✅ 訓練數據量: 18546
✅ 開發數據量: 105
✅ 測試數據量: 3856


In [10]:
# 建立詞典
word_to_idx = defaultdict(lambda: len(word_to_idx))
tag_to_idx = defaultdict(lambda: len(tag_to_idx))

# 特殊 Token
word_to_idx["<PAD>"] = 0
word_to_idx["<UNK>"] = 1
tag_to_idx["O"] = 0  # "O" == NOT entity

# Built the vol
for sentence in train_sentences:
    for word in sentence:
        _ = word_to_idx[word]

for label_seq in train_labels:
    for label in label_seq:
        _ = tag_to_idx[label]

print(f"✅ 詞彙數量: {len(word_to_idx)}, 標籤數量: {len(tag_to_idx)}")

✅ 詞彙數量: 22058, 標籤數量: 11


In [11]:
# 轉換數據（確保長度相同）
def encode_sentences(sentences, word_to_idx):
    return [[word_to_idx[word] if word in word_to_idx else word_to_idx["<UNK>"] for word in sent] for sent in sentences]

def encode_labels(labels, tag_to_idx):
    return [[tag_to_idx[tag] for tag in label_seq] for label_seq in labels]

train_x = encode_sentences(train_sentences, word_to_idx)
train_y = encode_labels(train_labels, tag_to_idx)

# 計算每個句子的長度
train_lengths = torch.tensor([len(seq) for seq in train_x], dtype=torch.long).to(device)

In [14]:
# 使用 `pad_sequence()` 來確保長度一致
train_x_tensor = [torch.tensor(seq, dtype=torch.long) for seq in train_x]
train_x_tensor = pad_sequence(train_x_tensor, batch_first=True, padding_value=word_to_idx["<PAD>"]).to(device)

train_y_tensor = [torch.tensor(seq, dtype=torch.long) for seq in train_y]
train_y_tensor = pad_sequence(train_y_tensor, batch_first=True, padding_value=tag_to_idx["O"]).to(device)

print(f"✅ `train_x_tensor` shape: {train_x_tensor.shape}")
print(f"✅ `train_y_tensor` shape: {train_y_tensor.shape}")

✅ `train_x_tensor` shape: torch.Size([18546, 204])
✅ `train_y_tensor` shape: torch.Size([18546, 204])


In [15]:
# BiLSTM-CRF 模型
class BiLSTM_CRF(nn.Module):
    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=1, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim, len(tag_to_ix))
        self.crf = CRF(len(tag_to_ix), batch_first=True)

    def forward(self, x, tags=None):
        embeds = self.embedding(x)

        # 計算有效的序列長度（忽略 padding）
        lengths = (x != 0).sum(dim=1).cpu()

        packed_input = pack_padded_sequence(embeds, lengths, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.lstm(packed_input)
        lstm_out, _ = pad_packed_sequence(packed_output, batch_first=True)

        emissions = self.fc(lstm_out)

        if tags is not None:
            # 只取有效長度的標籤，忽略 padding
            tags = tags[:, :emissions.shape[1]]
            return -self.crf(emissions, tags, reduction='mean')  # 計算損失
        else:
            return self.crf.decode(emissions)  # Viterbi 解碼

In [18]:
# 設置模型參數
embedding_dim = 100
hidden_dim = 128
epochs = 3

# 建立模型
model = BiLSTM_CRF(len(word_to_idx), tag_to_idx, embedding_dim, hidden_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 訓練 BiLSTM-CRF
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x, y in zip(train_x_tensor, train_y_tensor):
        x, y = x.to(device), y.to(device)  # 確保 x 和 y 在 GPU
        model.zero_grad()

        # 確保 y 是 2D
        if y.dim() == 1:
            y = y.unsqueeze(0)  # 變成 (1, seq_len)

        # 只取 `y` 的有效部分
        y = y[:, : (x != word_to_idx["<PAD>"]).sum().item()]

        loss = model(x.unsqueeze(0), y)  # 傳入 x 和 y
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

/usr/local/lib/python3.11/dist-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:530.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


Epoch 1, Loss: 109778.4912
Epoch 2, Loss: 89081.1827
Epoch 3, Loss: 82817.5355


In [19]:
# 先建立一個從 index 到標籤的映射字典
idx_to_tag = {idx: tag for tag, idx in tag_to_idx.items()}

# 將模型設置為評估模式
model.eval()

# 開啟輸出檔案以寫入預測結果
with open("dev.output", "w", encoding="utf-8") as fout:
    # 逐句進行預測
    for sentence in dev_sentences:
        # 將句子轉換成索引表示，如果找不到則用 <UNK>
        sent_idx = [word_to_idx[word] if word in word_to_idx else word_to_idx["<UNK>"] for word in sentence]
        x_tensor = torch.tensor(sent_idx, dtype=torch.long).unsqueeze(0).to(device)

        # 使用模型進行預測，返回的是列表，每個句子一個列表
        pred = model(x_tensor)[0]
        # 將預測的 index 轉換回標籤
        pred_tags = [idx_to_tag[p] for p in pred]

        # 寫入每個 token 與預測標籤
        for token, tag in zip(sentence, pred_tags):
            fout.write(f"{token}\t{tag}\n")
        # 句子之間以空行分隔
        fout.write("\n")

print("✅ 預測結果已寫入 dev.output")


✅ 預測結果已寫入 dev.output


In [20]:
!perl /content/drive/MyDrive/A2-data/A2-data/evalIOB2.pl /content/drive/MyDrive/A2-data/A2-data/dev.answers ./dev.output



[Tagging Performance]
# of tags: 2800,	 correct tags: 2363,	 correct IOBs: 2446
precision with class info: 0.8439,	 w/o class info: 0.8736

[Object Identification Performance]
# of OBJECTs: 203,	 ANSWERs: 238.

# (recall / precision / f-score) of ...
     FULLY CORRECT answer with class info: 88 (0.4335 / 0.3697 / 0.3991),
    correct LEFT boundary with class info: 104 (0.5123 / 0.4370 / 0.4717),
   correct RIGHT boundary with class info: 107 (0.5271 / 0.4496 / 0.4853),


[<cell_line> Identification Performance]
# of OBJECTs: 2,	 ANSWERs: 10.

# (recall / precision / f-score) of ...
     FULLY CORRECT answer with class info: 0 (0.0000 / 0.0000 / 0.0000),
    correct LEFT boundary with class info: 1 (0.5000 / 0.1000 / 0.1667),
   correct RIGHT boundary with class info: 0 (0.0000 / 0.0000 / 0.0000),


[<protein> Identification Performance]
# of OBJECTs: 124,	 ANSWERs: 153.

# (recall / precision / f-score) of ...
     FULLY CORRECT answer with class info: 63 (0.5081 / 0.4118 / 0.4549),


In [21]:
# 假設你已經有 read_test_data 以及 test_sentences
# 並且 idx_to_tag 也已建立

model.eval()  # 設定模型為評估模式

with open("test.output", "w", encoding="utf-8") as fout:
    for sentence in test_sentences:
        # 將句子中的每個 token 轉換成索引，若找不到則用 <UNK>
        sent_idx = [word_to_idx[word] if word in word_to_idx else word_to_idx["<UNK>"] for word in sentence]
        x_tensor = torch.tensor(sent_idx, dtype=torch.long).unsqueeze(0).to(device)

        # 使用模型進行預測，返回一個 list (每個句子一個 list)
        pred = model(x_tensor)[0]
        # 將預測的 index 轉換回標籤
        pred_tags = [idx_to_tag[p] for p in pred]

        # 輸出格式：每行一個 token 與預測標籤，以 tab 分隔，句子之間空一行
        for token, tag in zip(sentence, pred_tags):
            fout.write(f"{token}\t{tag}\n")
        fout.write("\n")

print("✅ 預測結果已寫入 test.output")


✅ 預測結果已寫入 test.output


In [22]:
!perl /content/drive/MyDrive/A2-data/A2-data/evalIOB2.pl /content/drive/MyDrive/A2-data/A2-data/test_answers/test.answers ./test.output



[Tagging Performance]
# of tags: 101039,	 correct tags: 86634,	 correct IOBs: 89229
precision with class info: 0.8574,	 w/o class info: 0.8831

[Object Identification Performance]
# of OBJECTs: 8662,	 ANSWERs: 8924.

# (recall / precision / f-score) of ...
     FULLY CORRECT answer with class info: 4543 (0.5245 / 0.5091 / 0.5167),
    correct LEFT boundary with class info: 5128 (0.5920 / 0.5746 / 0.5832),
   correct RIGHT boundary with class info: 5503 (0.6353 / 0.6167 / 0.6258),


[<cell_type> Identification Performance]
# of OBJECTs: 1921,	 ANSWERs: 1451.

# (recall / precision / f-score) of ...
     FULLY CORRECT answer with class info: 944 (0.4914 / 0.6506 / 0.5599),
    correct LEFT boundary with class info: 984 (0.5122 / 0.6782 / 0.5836),
   correct RIGHT boundary with class info: 1162 (0.6049 / 0.8008 / 0.6892),


[<RNA> Identification Performance]
# of OBJECTs: 118,	 ANSWERs: 160.

# (recall / precision / f-score) of ...
     FULLY CORRECT answer with class info: 68 (0.5763 / 